# Config

I recommand you run cell by cell...

In [ ]:
import requests
import math
import json
import time

api_url = 'http://localhost:3000'

# The basename of new playlists (basename-1, basename-2, ...)
basename = 'likelist'

# From browser's devtools(F12 in chrome) -> Network tab -> Request Headers (under any get like /login/status)
cookie = ""

# New playlist's max size
max_size = 1000

# Utils

In [ ]:
# Remove duplicated while keeping order.
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

# Main

In [ ]:
# Wrapper of NeteaseCloudMusicAPI
class API:
    def __init__(self, url) -> None:
        self.url = url
        self.cookie = ''

    def __get(self, url):
        return requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36', 'Cookie': self.cookie})

    def login_status(self):
        url = self.url + '/login/status'
        r = self.__get(url)

        return r
    
    def likelist(self, uid=''):
        url = self.url + '/likelist?uid={}'.format(uid)
        return self.__get(url)
    
    def playlist_track_all(self, id, limit='', offset=0):
        url = self.url + '/playlist/track/all?id={}&limit={}&offset={}'.format(id, limit, offset)
        r = self.__get(url)
        return r
    
    def _playlist_track_all(self, id, total):
        ret = []
        times = math.ceil(total / 1000)
        for i in range(times):
            r = self.playlist_track_all(id, 1000, 1000 * i)
            d = json.loads(r.text)
            ret += d['songs']
        return ret
    
    def user_playlist(self, uid):
        url = self.url + '/user/playlist?uid={}'.format(uid)
        r = self.__get(url)
        return r

    def refresh(self):
        url = self.url + '/login/refresh'
        r = self.__get(url)
        return r
    
    def check_music(self, id, br=''):
        url = self.url + '/check/music?id={}&br={}'.format(id, br)
        r = self.__get(url)
        return r
    
    def playlist_create(self, name, privacy='', type=''):
        url = self.url + '/playlist/create?name={}&privacy={}&type={}'.format(name, privacy, type)
        r = self.__get(url)
        return r

    def playlist_tracks(self, op, pid, tracks: str):
        url = self.url + '/playlist/tracks?op={}&pid={}&tracks={}'.format(op, pid, tracks)
        r = self.__get(url)
        return r

api = API(api_url)

In [ ]:
api.cookie = cookie

In [ ]:
r_login_status = api.login_status()
try:
    d_login_status = json.loads(r_login_status.text)
    profile = d_login_status['data']['profile']
    uid = profile['userId']
    nickname = profile['nickname']
    print('Login account profile:\nuid: {}\nnickname: {}'.format(uid, nickname))
except (KeyError, TypeError):
    print('You must login first.')
    print(d_login_status)

In [ ]:
# Get the size of likelist
r_likelist = api.likelist()
d_likelist = json.loads(r_likelist.text)
c_song = len(d_likelist['ids'])
print('total song count', c_song)

In [ ]:
# Check size
from math import ceil
if c_song <= max_size:
    print('The size of likelist({}) is lower than threshold({}).\nNo need to split.'.format(c_song, max_size))

In [ ]:
# Get playlists
r_user_playlist = api.user_playlist(uid)
d_user_playlist = json.loads(r_user_playlist.text)

In [ ]:
# Get likelist id
likelist = d_user_playlist['playlist'][0]
id_likelist = likelist['id']

In [ ]:
# Get all the tracks of likelist
likelist_track_all = api._playlist_track_all(id_likelist, c_song)

In [ ]:
# Split tracks
def split_tracks(tracks):
    """Assume the tracks are in time descent order."""

    ret = {}
    c_newlist = ceil(len(tracks) / max_size)
    l_name_newlist = [basename + '-{}'.format(x+1) for x in range(c_newlist)]
    print('names:\t', l_name_newlist)

    r_tracks = tracks[::-1]
    for i, n in enumerate(l_name_newlist):
        ret[n] = r_tracks[i * max_size : (i + 1) * max_size]

    print('counts:\t', [len(dict_newlists[n]) for n in l_name_newlist])
    
    return ret

dict_newlists = split_tracks(likelist_track_all)

In [ ]:
# Create playlists
def create_playlist(name, songs):
    """

    Warn
    ---
    Empty target playlist is recommended.
    """
    r_playlist_create = api.playlist_create(name)
    d_playlist_create = json.loads(r_playlist_create.text)
    try:
        id_playlist = d_playlist_create['playlist']['id']
    except KeyError:
        print('Failed create playlist {}'.format(name))
        print(d_playlist_create)
        return False
    
    ids_song = [str(song['id']) for song in songs[::-1]]
    ids_song_ = f7(ids_song)
    diff = len(ids_song) - len(ids_song_)
    if diff:
        print('{} duplicated songs in {}, removed'.format(diff, name))
    ids_song = ids_song_

    r_playlist_tracks_add = api.playlist_tracks('add', id_playlist, ','.join(ids_song))
    d_playlist_tracks_add = json.loads(r_playlist_tracks_add.text)

for name, songs in dict_newlists.items():
    create_playlist(name, songs)
    time.sleep(3)
